In [1]:
import soundfile # to read audio file
import numpy as np
import pandas as pd
import librosa # to extract speech features
import glob
import os
import pickle 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model, to_categorical

In [2]:
def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        if (len(X.shape) != 1):
            if (X.shape[1] == 2):
                X = X[:,0]
            else:
                return [None]
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

In [3]:
## codes in data
int2emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# only files with these emotion labels are used
emotions_dict ={
    "angry": 0,
    "sad": 1,
    "neutral": 2,
    "happy": 3
}

category_to_emotion = {}

for key in emotions_dict.keys():
    value = emotions_dict[key]
    category_to_emotion[value] = key

In [4]:
def load_data(test_size=0.2, max_amount=10000):
    i = 0
    X, y = [], []
    amount = 0
    for file in glob.glob("ravdess/Audio*/Actor_*/*.wav"):
        if (amount >= max_amount):
            break
        
        basename = os.path.basename(file)
        
        emotion = int2emotion[basename.split("-")[2]]
        allowed_emotions = emotions_dict.keys()
        
        if emotion not in allowed_emotions:
            continue
        emotion_category = emotions_dict[emotion]
        
        features = extract_feature(file, mfcc=True, chroma=True, mel=True)
        
        ## there seems to be some unusable data so lets get rid of those
        if (features[0] == None):
            continue
        
        
            
        X.append(features)
        y.append(emotion_category)
        amount = amount + 1
        
    return train_test_split(np.array(X), y, test_size=test_size, random_state=7)

In [5]:
max_amount_of_data = 100000
X_train, X_test, y_train, y_test = load_data(test_size=0.25, max_amount=max_amount_of_data)

In [6]:

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)


y_train = np.asarray(y_train)
y_train = y_train.astype(np.float32)
y_test = np.asarray(y_test)
y_test = y_test.astype(np.float32)

nb_classes = len(emotions_dict.keys())

y_train = to_categorical(y_train, nb_classes)
y_test = to_categorical(y_test, nb_classes)

In [7]:
print("[+] Number of training samples:", X_train.shape[0])
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape[0])
# number of features used
# this is a vector of features extracted 
# using extract_features() function
print("[+] Number of features:", X_train.shape[1])


[+] Number of training samples: 987
[+] Number of testing samples: 329
[+] Number of features: 180


In [8]:
print("[+] X_train first value's first 5 values", X_train[0][0:5])
print("[+] y_train first value", y_train[0])

[+] X_train first value's first 5 values [-665.39246     97.90721     -4.6756134   13.205296     4.945498 ]
[+] y_train first value [0. 1. 0. 0.]


In [9]:
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)
x_traincnn.shape

(987, 180, 1)

In [10]:
inputs = keras.Input(shape=(x_traincnn.shape[1], x_traincnn.shape[2]))
x = layers.Conv1D(128, 5, activation="relu", padding="same")(inputs)

x = layers.Dropout(rate=0.1)(x)

x = layers.MaxPooling1D(pool_size=(8))(x)
x = layers.Conv1D(128, 5, activation="relu", padding="same")(x)
x = layers.Dropout(rate=0.1)(x)
x = layers.Flatten()(x)


output_units = len(emotions_dict.keys())
outputs = layers.Dense(units=output_units, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs,
                    name="mlp_model")
opt = keras.optimizers.RMSprop(lr=0.00005, rho=0.9, epsilon=None, decay=0.0)

model.compile(loss='categorical_crossentropy', 
              optimizer=opt, 
              metrics=['accuracy'])
print(model.summary())

ModuleNotFoundError: No module named 'keras'

In [ ]:
epochs = 200

history = model.fit(x_traincnn, y_train, 
                    epochs=epochs, 
                    batch_size=32,
                    verbose=2)

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(history.epoch,history.history['loss'])
plt.title('loss')

plt.figure(figsize=(5,3))
plt.plot(history.epoch,history.history['accuracy'])
plt.title('accuracy');

In [ ]:
scores = model.evaluate(x_testcnn, y_test, verbose=2)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
from sklearn.metrics import confusion_matrix
predictions = model.predict(x_testcnn)
print('Confusion matrix (rows: true classes; columns: predicted classes):'); print()
cm=confusion_matrix(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1), labels=list(range(4)))
print(cm); print()

print('Classification accuracy for each class:'); print()
for i,j in enumerate(cm.diagonal()/cm.sum(axis=1)): print(f"{i} ({category_to_emotion[i]}): {j}")

# Testing on youtube videos

In [ ]:
from pydub import AudioSegment
import math
import shutil


In [ ]:
parent_dir = os.getcwd() + "\\test_yt_data"
temp_path = os.path.join(parent_dir, "tmp")
try:
    shutil.rmtree(temp_path)
except:
    None

In [ ]:
#

X = []
X_names = []
video_names = []
split_video_names = []
## split each video to 3 second videos and make predictions to those
for file in glob.glob("test_yt_data/*.wav"):
        file_splits = []
        basename = os.path.basename(file)
        sound = AudioSegment.from_file(file)
        three_seconds = 3 * 1000
        length = len(sound)
        splits_amount = math.ceil(length/three_seconds)
        i = 0
        parent_dir = os.getcwd() + "\\test_yt_data"
        temp_path = os.path.join(parent_dir, "tmp")
        os.mkdir(temp_path, 0o666)
        
        while (i < splits_amount):
            start_index = i * three_seconds
            end_index = start_index + three_seconds
            split = sound[start_index:end_index]
            basename_without_ext = basename.split(".")[0]
            split_video_name = f"{basename_without_ext}_split_{i}.wav"
            path = f"test_yt_data/tmp/{split_video_name}"
            split.export(path, format="wav")
            split_video_names.append(split_video_name)
            X_names.append(split_video_name)
            i = i + 1
        
        for file in glob.glob("test_yt_data/tmp/*.wav"):
            features = extract_feature(file, mfcc=True, chroma=True, mel=True)
            file_splits.append(features)
        shutil.rmtree(temp_path)
        video_names.append(basename)
        X.append(file_splits)

X = np.array(X)
X = X.astype(np.float32)

In [ ]:
print(X.shape)

In [ ]:
split_video_index = 0
for ids, values in enumerate(X):
    print("Predicting values for video '" + f"{X_names[ids]}")
    for index, val in enumerate(values):
        y_hat = model.predict( np.array( [val,] )  )[0]
        predicted = y_hat.argmax()

        emotion = category_to_emotion[predicted]
        print("Predicted emotion for split '" + f"{split_video_names[split_video_index]}" +  "' is:")
        print(emotion)
        print()
        split_video_index = split_video_index + 1